In [1]:
%load_ext Cython

In [9]:
def f(x):
    return x ** 2 - x

def integrate_f(a, b, N):
    s = 0
    dx = (b - a) / N
    for i in range(N):
        s += f(a + i * dx)
    return s * dx

In [13]:
%%timeit

integrate_f(0, 100, 10_000)

1.66 ms ± 52.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [11]:
%%cython

def f(x):
    return x ** 2 - x

def c_integrate_f(double a, double b, int N):
    cdef int i
    cdef double s
    cdef double dx
    s = 0
    dx = (b - a) / N
    for i in range(N):
        s += f(a + i * dx)
    return s * dx

In [14]:
%%timeit

c_integrate_f(0, 100, 10_000)

1.28 ms ± 2.42 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [18]:
round(1.28 / 1.66, 3)

0.771

In [29]:
import sysconfig
print(sysconfig.get_paths()["include"])

/opt/conda/include/python3.11


In [42]:
!cythonize -i Basic2D.pyx

Compiling /home/jovyan/work/Basic2D.pyx because it changed.
[1/1] Cythonizing /home/jovyan/work/Basic2D.pyx


In [43]:
!ls *.so

Basic2D.cpython-311-aarch64-linux-gnu.so


In [44]:
!g++ -shared -pthread -fPIC -fwrapv -O2 -Wall -fno-strict-aliasing \
-I/opt/conda/include/python3.11 -o Basic2D.cpython-311-aarch64-linux-gnu.so Basic2D.c \
-L/usr/local/lib -lgsl -lgslcblas -Wl,-rpath,/usr/local/lib

We had to do some debugging while figuring out how to link the cython version of the
random walk with gslcblas.so and we ended up checking the RPATH variable as follows:
```
$ readelf -d Basic2D.cpython-311-aarch64-linux-gnu.so | grep RPATH
 0x000000000000000f (RPATH)              Library rpath: [/opt/conda/lib]
```

In [45]:
import Basic2D
Basic2D.run_random_walks()

In [46]:
%timeit Basic2D.run_random_walks()

1.44 s ± 12.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [49]:
import numpy
print(numpy.get_include())

/opt/conda/lib/python3.11/site-packages/numpy/core/include


In [97]:
!cythonize -i Basic2D_pythonic.pyx

Compiling /home/jovyan/work/Basic2D_pythonic.pyx because it changed.
[1/1] Cythonizing /home/jovyan/work/Basic2D_pythonic.pyx


In [98]:
!g++ -shared -pthread -fPIC -fwrapv -O2 -Wall -fno-strict-aliasing \
-I/opt/conda/include/python3.11 -o Basic2D_pythonic.cpython-311-aarch64-linux-gnu.so Basic2D.c \
-L/usr/local/lib -lgsl -lgslcblas -Wl,-rpath,/usr/local/lib

In [99]:
import Basic2D_pythonic

%timeit Basic2D_pythonic.run_random_walks_pythonic()

ValueError: Buffer dtype mismatch, expected 'int_t' but got 'int'

In [53]:
%%writefile compile_cython.py
from Cython.Build import cythonize
from distutils.extension import Extension
import numpy

# Define your extension and include dirs
extensions = [
    Extension("Basic2D_pythonic", ["Basic2D_pythonic.pyx"], include_dirs=[numpy.get_include()])
]

# Use cythonize on the extension object
cythonize(extensions)

Overwriting compile_cython.py


In [66]:
!python compile_cython.py

In [67]:
!gcc -shared -pthread -fPIC -fwrapv -O2 -Wall -fno-strict-aliasing \
-I/opt/conda/include/python3.11 -I/opt/conda/lib/python3.11/site-packages/numpy/core/include \
-o Basic2D_pythonic.so Basic2D_pythonic.c \
-L/usr/local/lib -lgsl -lgslcblas -Wl,-rpath,/usr/local/lib \
-DNPY_NO_DEPRECATED_API=NPY_1_7_API_VERSION

In [81]:
import numpy as np

np.random.randint(0, 4, size=3)

array([0, 3, 2])